# Introduction to Python

## Basic Statistics Using Statsmodels

## Trainers

<font size="+0.5">Daniel Lewis</font>   
(<daniel.j.lewis@ons.gov.uk>)  
<font size="+0.5">Jhai Ghaghada</font>   
(<jhai.ghaghada@ons.gov.uk>)

## Table of Contents

<a href="#Background"><font size="+1">Background</font></a>
* Learning Objectives
* Statsmodels
* Quarterly Labour Force Survey Data
* Getting Started
    * `import` statsmodels
* <a href="#Exercise-1:-Explore-the-Data">Exercise 1: Explore the Data</a> 
    * Rows, Columns, Variables
    * Datatypes
* The `category` data type.

<a href="#Examining-Associations-in-Categorical-Variables"><font size="+1">Examining Associations in Categorical Variables</font></a>
* The $\chi^{2}$ (chi-squared) test
* Basic Weighting

<a href="#Examining-Differences-in-Numerical-Variables"><font size="+1">Examining Differences in Numerical Variables</font></a>
* The t-test
* Weighting
    * Descriptive Statistics
    * Confidence Intervals
    * t-tests
* <a href="#Exercise-2">Exercise 2: Hypothesis Testing</a>
    
<a href="#Regression-Analysis"><font size="+1">Regression Analysis</font></a>
* The Regression Equation
* The OLS Regression Model
* The OLS Model Summary
* Residuals
* Weighted Regression Model
* making Predictions

<a href="#Multiple-Linear-Regression"><font size="+1">Multiple Linear Regression</font></a>
* Multiple Linear Regression
* Weighted Multivariable Regression
* <a href="#Exercise-3">Exercise 3: Regression Models</a>


# Background

### Learning Objectives

The goal of this session is to:

* Become familiar with basic statistics using statsmodels
* Implement some common statistical tests
    * $\chi^{2}$ test for independence.
    * t-test for difference in means.
* Explore simple regression models
    * Simple linear regression (OLS regression; numeric outcome data)

### Statsmodels

Statsmodels is python's most fully featured statistical module. It provides access to a wide range of statistical models, tests and descriptive tools. We're going to use this module to conduct some simple statistical analysis using hypothesis tests and regresison analysis. However, it is worthwhile acknowledging that python's statistical capabilities are less well developed that statistical software like STATA or R, however they are developing rapidly. Currently, you will get a sense that performing some analyses is harder than it should be, simply because less time has been dedicated to the task than with more mature modules like pandas (which is itself still in active development).

In many ways, this makes both python generally and statsmodels in particular exiting to use, as functionality is continually evolving!

You can find the majority of undergraduate degree-level statistics in statsmodels: https://www.statsmodels.org/stable/index.html (scroll down the webpage for module contents), it's mostly in areas like complex survey analysis and sophisticated modelling approaches where it falls behind compared to STATA or R.

### Quarterly Labour Force Survey Data

In this vignette we'll be using a new dataset, the 'Quarterly Labour Force Survey, January - March, 2015'.

The purpose of the Labour Force Survey (LFS) is  to provide information on the UK labour market that can then be used to develop, manage, evaluate and report on labour market policies.

We're using the teaching dataset, available in STATA format from the UK Data Service: http://doi.org/10.5255/UKDA-SN-7912-1, this is open data under the Open Government License (OGL).

The teaching dataset is similar to the real dataset, except the number of records (rows) and variables (columns) has been reduced, some variables have been recoded to more aggregate representations, and a simple weight variable has been included.

### Getting Started

The code cell below does the standard import of pandas and matplotlib, and the new addition of statsmodels!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

%matplotlib inline

## Exercise 1: Explore the Data

Take an opportunity now to explore the new dataset, both using pandas, and by reading through the data dictionary provided in the data folder.

* Read the data into python using pandas - it's a STATA file, so think about how you're going to do that.
* How many records (rows) does the dataframe have?
* How many variables (columns) does the dataframe have?
* What are the datatypes of the available columns?


In [ ]:
# Enter code here.


In [ ]:
%load ../Solutions/Statistics/explore_data.py


### What are `category` data?

Most of the data use a datatype that we've not seen before - category. This is a special pandas datatype for storing categorical data - data that have a limited number of possible values (categories). Previously, our categorical data was simply represented as 'object' (text/string) data, however the category datatype formalises the distinction between text data, which can take an effectively unlimited set of values, and categories which usually have a fixed set of possible values.

Examples of categories in social surveys include: gender, social class, country of birth, marital status, employment type, and likert scale responses (i.e. 1-very likely... 5-very unlikely). Several of these categories are used in the Labour Force Survey you're currently exploring.

The categorical data type can store either nominal (unordered) or ordinal (ordered) data. Nominal categories have no specific order to them, and store information like ethnic group membership, by constrast ordinal categories have an implicit order, age groups, for instance, ascend for younger groups to older groups, while a categorical measure of disease severity might have three ordered categories: low, medium and high.

You can establish the unique categories, and whether those categories are ordered for a column by interrogating the column using the `.cat.` module properties:

In [ ]:
# Get category values
print("SEX categories")
print(lfs['SEX'].cat.categories)
# Get ordered or not
print("ordered =",lfs['SEX'].cat.ordered)
# As it happens, pandas isn't good at reading categorical values from stata data, the default is to set them all as ordered.

In [ ]:
# In the solution above (explore_data.py), we can use the order_categoricals=False.
# This sets the categoricals to nominal, which is mostly the case with LFS.

# Then we can set the age variable to be ordered.
lfs['AGEEULR'] = lfs['AGEEULR'].cat.as_ordered()
# Get category values
print("AGE categories")
print(lfs['AGEEULR'].cat.categories)
# Get ordered or not
print("ordered =",lfs['AGEEULR'].cat.ordered)

# Set NSECMJ3R to ordered too. NS-SEC 3.
# Unfortunately these order backwards as standard, so we have to reverse the order.
order = lfs['NSECMJ3R'].cat.categories[::-1]
lfs['NSECMJ3R'] = lfs['NSECMJ3R'].cat.set_categories(new_categories=order, ordered=True)

# Get category values
print("\nNSECMJ3R categories")
print(lfs['NSECMJ3R'].cat.categories)
# Get ordered or not
print("ordered =",lfs['NSECMJ3R'].cat.ordered)

# Also, the 'TOTHRS' variable should really be numeric, but it has some categorical properties 
# This means python has typed it as a category.
# We'll deal with this later when we want to use it!

When categories are ordered, `.min()`, `.max()` and `.sort_values()` have meaning, whereas when these categories were defined as text these functions just produced the alphabetical meaning, not the categorical (lexical) meaning. Similarly, when categories are unordered, these functions are meaningless and hence produce `TypeError`s.

The `.mode()` method has meaning regardless of whether the categories are ordered or unordered as this computes frequencies.

In [ ]:
# min and max of ordered categories
print("Minimum Age is", lfs['AGEEULR'].min())
print("Maximum Age is", lfs['AGEEULR'].max())
print("The modal Age is", lfs['AGEEULR'].mode()[0]) # remember, mode could be several categories.

In [ ]:
# min of unordered categories
lfs['SEX'].min()

Category variables are largely intuitive to use, although they can have some caveats, for more information you can check out the extensive pandas documentation on categories: https://pandas.pydata.org/pandas-docs/stable/categorical.html

# Examining Associations in Categorical Variables

Statsmodels supports analyses of contingency tables, the process is very straightforward:
1. Use pandas to make a cross-tabulation of categorical variables.
2. Pass the cross-tabulation to statsmodels.
3. Call the relevant statistical functions to explore the crosstab.

Let's start very simply, and look at the association between full-time and part-time work by sex.

Our **null hypothetis** would be that there is no association between sex and type of work (part-time or full-time).

Our **alternative hypothesis** assumes that there is an association.

In fact, my expectation would be that women would be more likely to be in part-time work than men. Let's see if this can be verified:

In [ ]:
# First filter data for only those in full- and part-time work.
# Effectively, exclude those for whom this question doesn't apply (n= 5150) or didn't answer (n=15)

# filter the data
ftptwork = lfs[lfs['FTPTWK'].isin(['Full-time','Part-time'])]
# make the cross-tab
tab = pd.crosstab(ftptwork['SEX'],ftptwork['FTPTWK'])
tab

In [ ]:
# Now pass this cross-tab to a statsmodels table object
table = sm.stats.Table(tab)

Using `table` we can now run statistical tests. As both sex and work type are nominal (i.e. unordered) we can use Pearson's $\chi^{2}$ statistic (Chi-squared test):

In [ ]:
# Run the chi-squared test and store output in rslt
rslt = table.test_nominal_association()
# Get the chi-squared statistic
print("chi-squared statistic =", rslt.statistic)
# Get the p-value
print("p = ", rslt.pvalue)

In this case, there is a very strong association between sex and work type. The reported $\chi^{2}$ statistic is very large, and the p-value is effectively 0, meaning that this association is very unlikely to have occured by chance.

We can see this in the original table, by looking at the row percentages:

In [ ]:
pd.crosstab(ftptwork['SEX'],ftptwork['FTPTWK'],normalize='index')

It is evident above that a much greater proportion of male respondents work full-time than do female respondents, for those in work, 85% of men against only 55% of women work full-time.

## Basic Weighting

The Labour Force Survey is not a simple random sample, it has a complex survey design (like most contemporary surveys). The LFS is a geographically stratified random sample, with participants drawn at random from the postcode address file (PAF; a list of c.97% of private households in GB) and supplemented with some other methods. The survey approach means that members of some population groups have a different chance of being selected to participate in the survey than members of other groups. It also has some differences in response rate between population groups, some people were more likely to actually choose to take part than others. This means that statistics produced for the raw data are not necessarily generalisable to the population as a whole.

In order to produce information that does generalise to the general population, or the population of interest for a particular survey, weights are produced that adjust the influence of each respondent relative to the overall population. In the LFS, cross-sectional weights account for age, sex and region of participants, adjusting to Census-based estimates of population. It is also possible to use the LFS as a longitudinal survey, however this application is more eadvanced.

The Labour Force Survey teaching dataset has two weights: 'PWT14' and 'PWT14R':
* PWT14 is a 'grossing rate', meaning that applying it produces population totals, it multiplies the data up to the estimated population of England and Wales at the time.
* PWT14r is a relative weight, it has the same effect as a 'grossing weight', but makes sure that the total produced is the same size as the sample. This is useful for statistical inference where sample standard error is a function of sample size. Using the grossing weight would make the statistics produced seem much less uncertain that they really are.

Because this is a teaching dataset, and not the 'real' LFS survey for the dates covered, the weights here are more a teaching and learning tool than a verifiable and accurate component of the data, so take any conclusions generated with a pinch of salt!

We'll use the relative weights to adjust the chi-squared test we've just performed:

In [ ]:
# Remake the weights tab by including the weights as a 'values' variable and summing those weights.
tabW = pd.crosstab(ftptwork['SEX'],ftptwork['FTPTWK'],ftptwork['PWT14R'],aggfunc='sum')
# Compare these totals to the unweighted totals. They're actually pretty close for men, less so for women.
tabW

In [ ]:
# now pass weight_tab to statsmodels
tableW = sm.stats.Table(tabW)
# Get the stats
rsltW = tableW.test_nominal_association()
# Get the chi-squared statistic
print("chi-squared statistic =", rsltW.statistic)
# Get the p-value
print("p = ", rsltW.pvalue)

Although the statistic goes down a little, the association is still extremely strong and very unlikely to have occured by chance.

Note though, that this example of weighting a $\chi^{2}$ test is known as the 'uncorrected $\chi^{2}$', and should be used cautiously. It is simply the normal $\chi^{2}$ statistic with weighted cell proportions. Complex survey designs may violate some assumptions of the normal $\chi^{2}$ statistic, which is why the 'adjusted $\chi^{2}$' statistic is preferred when working with surveys that require weighting for generalisability.

Unfortunately, statsmodels doesn't currently implement $\chi^{2}$ for survey data, however, if you need to produce this statistic you can use the 'survey' library in R and the svytable function, or svy in STATA with tab.

# Examining Differences in Numerical Variables

We can also use statsmodels to examine numerical responses by categories. In the Labour Force Survey dataset we only have one numerical variable - 'TOTHRS' - which is the total hours works in the survey reference week by each respondent.

However, we need to filter and convert this variable to numeric first in order to make proper use of it.

In [ ]:
# First, filter out the 'Does not apply' (n=8727) and 'No answer' (n=230) responses.
lfs_hours = lfs[~lfs['TOTHRS'].isin(['Does not apply','No answer'])]
# Now replace '97 or more' (n=20) with '97', so it can be converted to a number.
lfs_hours = lfs_hours.replace({'TOTHRS':{'97 or more':'97'}})
# Now convert 'TOTHRS' to numeric
lfs_hours['TOTHRS'] = lfs_hours['TOTHRS'].astype(float)

Let's have a quick look at the distribution of hours in this dataframe using a histogram.

In [ ]:
f, ax = plt.subplots(figsize=(10,6))
ax.hist(lfs_hours['TOTHRS'],bins=20,color='seagreen')
ax.set_ylabel('Frequency')
ax.set_xlabel('Hours Worked in Reference Week')
ax.yaxis.grid(color='0.741')

Now that the 'TOTHRS' column is numeric, we can produce any number of descriptive statistics for it.

In [ ]:
print("Mean hours worked:",lfs_hours['TOTHRS'].mean(),"\n")
print("Median hours worked:",round(lfs_hours['TOTHRS'].median(),2))

These values look plausible. We can also use groupby to look at statistics by different groups:

In [ ]:
lfs_hours.groupby('SEX')['TOTHRS'].mean()

So, according to our groupby, the typical woman works around 10 hours per week less than the typical man.

However, this is sample data, we need to perform a statistical test to confirm whether the difference we observe here is likely to be a true difference in the general population.

So, lets compare mean hours worked for men and women using a statistical test - a t-test.

Again, the **null hypothesis** would be that there is no difference in hours worked between men and women.
Whereas, the **alternative hypothesis** is that there is a difference.

In [ ]:
# First, group the data
sex = lfs_hours.groupby('SEX')['TOTHRS']
# Now pass the two groups to statsmodels
t_test = sm.stats.ttest_ind(sex.get_group('Male'),sex.get_group('Female'))
# Now print the test statistic and p-value
print("t statistic =",round(t_test[0],2))
print("p-value =",round(t_test[1],3))

As we can see from the t-test, the means of the group are statistically significantly different, and very unlikely to have been different by chance. This suggests that there is likely to be a true difference in the population in terms of hours worked between men and women.

This difference is evident in the histograms of the two groups.

In [ ]:
f, ax = plt.subplots(figsize=(10,6))

ax.hist(sex.get_group('Male'),alpha=0.33, color='r',bins=20, density=True, label='Men')
ax.hist(sex.get_group('Female'),alpha=0.33, color='b', bins=20, density=True, label='Women')
ax.set_xlabel("Hours worked in reference week")
ax.set_ylabel("Density")
ax.legend(fontsize=12);

## Weighting

Unlike with categorical data, statsmodels allows for weights for descriptive statistics and t-tests.

### Weighted Descriptive Statistics

Descriptive statistics that are weighted can be obtain through statsmodels using the `.DescStatsW()` function.

Weighted descriptive statistics are shown below for the 'TOTHRS' variable.

In [ ]:
# Pass the data, and the weights to the DescrStatsW function.
W = sm.stats.DescrStatsW(lfs_hours['TOTHRS'],lfs_hours['PWT14R'])

In [ ]:
# Get the mean hours worked for weighted and unweighted data
w_mean = W.mean
uw_mean = lfs_hours['TOTHRS'].mean()
print("Estimated mean hours worked in the population are:",round(w_mean,2))
print("Estimated mean hours worked in the sample are:",round(uw_mean,2))

Accounting for survey weights gives an estimate of the mean hours that is slightly higher than the unweighted sample estimate.

A particularly useful output that `.DescrStatsW()` also provides is the confidence interval for the mean.

A confidence interval is the interval within which we would expect the true population mean to lie with given (e.g. 95%) confidence. As such it measures the uncertainty in a sample mean. (NB This definition of a confidence interval is not *exact* but will do for now.)

Work out the 95% confidence interval around the mean for hours worked.

In [ ]:
lower, upper = W.tconfint_mean()
ci = "(" + str(round(lower,2))+", "+str(round(upper,2)) +")"
print("The mean and 95% confidence interval for hours worked are:", round(w_mean,2),ci)

Note that if we wanted to get unweighted means and confidence intervals, we can use `.DescStatsW()` without passing any weights, this effectively sets all weights to 1 (i.e. all observations weighted equally).

In [ ]:
d = sm.stats.DescrStatsW(lfs_hours['TOTHRS']) # note, no weights.
print(round(d.mean,2)) # same as pandas
print(d.tconfint_mean(alpha=0.01)) # NB 99% confidence interval.

If we want to get the weighted means and confidence intervals for groups, we can to do something similar to how we implemented the t-test earlier, using `.groupby()` and .`get_group()`.

In [ ]:
# First, group the data including the weight this time.
sexW = lfs_hours.groupby('SEX')[['TOTHRS','PWT14R']]

# Split the groups into separate weight functions.
W_male = sm.stats.DescrStatsW(sexW.get_group('Male')['TOTHRS'],sexW.get_group('Male')['PWT14R'])
W_female = sm.stats.DescrStatsW(sexW.get_group('Female')['TOTHRS'],sexW.get_group('Female')['PWT14R'])

# Now get the weighted means and std deviations
print("Men:",W_male.mean,W_male.tconfint_mean())
print("Women:",W_female.mean,W_female.tconfint_mean())

In [ ]:
# Here's a reminder of the unweighted means by sex
lfs_hours.groupby('SEX')['TOTHRS'].mean()

Evidently, weighting causes the estimate of weekly hours to rise slightly for women and more or less stay the same for men.

In the example above, we had to run the process twice, once for each group of data, this is because Statsmodels doesn't have a groupby function like pandas. However, we can use python to help us avoid having to repeat ourselves too much. A `for` loop approach might look like this:

In [ ]:
# First, group the data including the weight this time.
sexW = lfs_hours.groupby('SEX')[['TOTHRS','PWT14R']]

# iterating over a groupby object produces a tuple: (groupname, sub dataframe)
for group, df in sexW:
    # make DescrStatsW object
    W = sm.stats.DescrStatsW(df['TOTHRS'],df['PWT14R'])
    print(group,W.mean,W.tconfint_mean())

The above approach is particularly useful as the number of groups increases!

### Weighted t-test

Now that we can produce weighted descriptive statistics, lets also use a weighted t-test so that we can generalise the results of inference to the population!

All we need to do to weight our t-test is to specify the weights parameter, which takes a tuple of weights.

In [ ]:
# First, group the data again including the weights.
sexW = lfs_hours.groupby('SEX')[['TOTHRS','PWT14R']]

# make weights tuple
weights = (sexW.get_group('Male')['PWT14R'],sexW.get_group('Female')['PWT14R'])

# Now pass to t-test
tW = sm.stats.ttest_ind(sexW.get_group('Male')['TOTHRS'],sexW.get_group('Female')['TOTHRS'],weights=weights)

# Now print the test statistic and p-value
print("t statistic =",round(tW[0],2))
print("p-value =",round(tW[1],3))

Using the weighted t-test, the t statistic goes down slightly compared to the unweighted t-test, however evidence for a true difference in hours worked between men and women in the population remains extremely strong, and is unlikely to have occured by chance. On average, women are likely to work fewer hours than men.

## Exercise 2

Now that you've had a chance to perform some hypothesis tests using statsmodels, try answering the following questions:

1. Is there evidence of an association between being an employee or being self-employed for men and women?
    * Use the 'SEX' and 'STAT3R' variables, filter so that 'STAT3R' only includes 'Employee' and 'Self-employed'
    * Create a cross-tabulation of SEX and STAT3R, normalize by row ('index') - what do you notice?
    * Now pass a frequency crosstab to statsmodels, what is the $\chi^{2}$ statistic and p-value?
    * What do you conclude about the relationship between sex and employee status?
2. Is there evidence of an association between being an employee or being self-employed by ethnic group?
    * Use the 'ETHUK7R' and 'STAT3R' variables, filter so that 'STAT3R' only includes 'Employee' and 'Self-employed', and 'ETHUK7R' excludes respondents who gave 'No answer'.
    * Create a cross-tabulation of ETHUK7R and STAT3R, normalize by row ('index') - what do you notice?
    * Calculate the $\chi^{2}$ statistic and p-value.
    * Investigate the drivers of the $\chi^{2}$ statistic with the `.chi2_contribs` property of the table.
    * What can you conclude about the association between ethnic group and employee status?
3. Is there evidence that the self-employed work, on average, more hours per week than employees?
    * Filter the data so that 'STAT3R' only includes 'Employee' and 'Self-employed', and 'TOTHRS' excludes 'Does not apply' and 'No answer'.
    * Make sure the 'TOTHRS' is a numeric variable.
    * Use `.groupby()` to get the unweighted mean weekly hours worked for employees and the self-employed.
    * What do you make of the difference between the data?
    * Create weighted mean hours worked and their 95% confidence intervals for employees and the self-employed.
    * Run a weighted t-test to explroe whether there is likely a true difference between employees and the self-employed in terms of wekely hours worked in the population.
    * What can you conclude about the difference in hours worked by employees and the self-employed?
   

In [ ]:
# Start you code in this cell


In [ ]:
# Question 1
%load ../Solutions/Statistics/ex2_q1.py


In [ ]:
# Question 2
%load ../Solutions/Statistics/ex2_q2.py


In [ ]:
# Question 3
%load ../Solutions/Statistics/ex2_q3.py


# Regression Analysis

This section will demonstrate the application of linear regression using statsmodels, however it won't go over the statistical specifics and theory of regression in much detail, so some prior knowledge is assumed.

In their simplest form, linear regression models are essentially machines for making means. We can demonstrate this by using a model to reproduce the means observed in the `.groupby()` statement used earlier to look at weekly hours worked by men and women.

In [ ]:
model = sm.formula.ols(formula='TOTHRS~SEX',data=lfs_hours).fit()
model.summary()

In the above model, we can see that the 'Intercept' is 36.4782 hours, this is the mean hours worked by men, and the equivalent for women is 36.4782 (intercept) - 9.6418 (coef for SEX == Female) = 26.8364 hours.

Note below that these values are the same as given by the `.groupby()`:

In [ ]:
lfs_hours.groupby('SEX')['TOTHRS'].mean()

Perhaps it doesn't immediately appear that useful to be able to estimate means using a regression model. Particularly if we can already do it using a simple descriptive analysis.

However, the beauty of the regression model is that it allows you to estimate the mean of an outcome for a given variable, while controlling for the effect of other variables. This allows you to estimate the independent effect of a particular variable, and hence get a sense of that variable's likely importance. This is useful as it can give you a sense of the likely affect of changing that variable (assuming that variable is 'modifiable') on the outcome of interest.

Before we get into multivariate linear regression, lets make sure we understandthe process for producing the simple example given above.

## The Regression Equation

We are using statsmodels in 'formula' mode, this means that we can specify the model we wish to run as a short text equation, and give it to statsmodels to interpret. if you have any experience doing this in R, you might be pleased to learn that the equations in R and the equations in statsmodels are the same! In the example above, our formula looked like this:

'TOTHRS~SEX'

Here, the ~ (tilde) means 'is modelled by', so the full equation effectively says: 'total hours is modelled by respondent sex.'

## The OLS Regression Model

The objective of the OLS regression model that we fit is to find the best linear solution to the regression equation such that the resultant model explains as much of the variance in 'TOTHRS' as possible. In this case, this is achieved when:

'TOTHRS' = 36.4782 - 9.6418 * 'SEX'

In order to create the model we assign:
```python
sm.formula.ols(formula='TOTHRS~SEX',data=lfs_hours)
```
to a variable, where the formula entries reference columns given in the dataframe `lfs_hours`.

We can then `.fit()` this model and calculate the intercept and coefficients that minimises the residual variation in the outcome.

## The OLS Model Summary

Having specified the formula and fit the model, we can see a summary of the results by calling the `.summary()` function of the fitted model.

Let's have another look at the summary:

In [ ]:
model.summary()

The summary gives us a range of diagnostic information about the model we've fitted, split into three tables. Important things to note include:
* Top Table - model fit info.
    * R-squared/ Adj. R-squared - The proportion of the variance in the dependent variable ('TOTHRS') explained by the model.
    * No. of Observations
    * General info - date and time that model was run, type of model etc.
    * Other statistical info inc. F test, AIC and BIC, log-likelihood.
* Middle Table - an important table!
    * coef = coefficient estimates for the intercept and explanatory variables.
    * std err = standard errors (i.e. estimate of the uncertainty) of the coefficient estimates.
    * t = t-statistic for the t-test comparing whether the coefficient is different to 0.
    * P>|t| = p-value for the t statistics, giving significance of coefficient.
    * [0.025 0.975] = 95% confidence interval around the coefficient estimate.
* Bottom table - Diagnostics
    * Jarque-Bera, Omnibus: test normality of residuals.
    * Condition Number: test for multicollinearity.
    * Durbin-Watson: test for autocorrelation.

We can get specific parameters directly from the `model` object if we want to:

In [ ]:
print("Intercept = ",model.params['Intercept'])
print("(Sex == Female) coef. = ",model.params['SEX[T.Female]'])
print("R^2 = ", model.rsquared)

In [ ]:
model = sm.formula.ols(formula='TOTHRS~SEX - 1',data=lfs_hours).fit()
model.summary()

## Residuals



The diagnostic information included in the regression model summary tells us that our residuals are likely to deviate from normality, violating an assumption of the regression model. The Jarque-Bera test looks at the skew and kurtosis of the residuals and compares them to the expected values for a normal distribution (0; no skew or kurtosis), our model residuals exhibit both skew and kurtosis meaning that the test concludes that the residuals are unlikely to be normal.

We can go further, and plot the residuals in order to understand what is going on in our modelled residuals.

Model residuals are simply the sample estimate of the error for each observation, that is, they are the difference between the observed value we measured (hours worked) and the modelled estimate of that value.

Currently, we have a very simple model, in which hours worked is modelled only by respondent sex, which explains 7.5% of the variation, so we might anticipate some residuals to be large.

In [ ]:
# Typically, we might compared the fitted values (predictions) to the residuals using a scatter plot
# Because our explanatory variable is binary (sex), there are only 2 unique fitted values: 36.48 (men), 26.84 (women)
# Therefore a boxplot will work better

# Get the model's residual errors
resid = model.resid

# add to lfs_hours datframe
lfs_hours['residuals'] = resid

# group residuals by sex
res_grp = lfs_hours.groupby('SEX')['residuals']

# Make a plot
f, [ax1,ax2] = plt.subplots(1,2,figsize=(15,7),sharey=True)
# Now use matplotlib's boxplot to compare the residuals
ax1.boxplot([res_grp.get_group('Male'),res_grp.get_group('Female')], labels=['Male','Female'])

# Alternatively, try statsmodels 'violinplot'
sm.graphics.violinplot([res_grp.get_group('Male'),res_grp.get_group('Female')],labels=['Male','Female'], ax=ax2)

ax1.set_title("Box Plot of Residuals by Sex")
ax2.set_title("Violin Plot of Residuals by Sex");

The plots above show the standard boxplot of residuals by sex, on the left, and statsmodels 'violinplot' on the right. A violin plot is a boxplot with some additional information added about the data using kernel density estimation. In this sense, the shape of the green volumes give you a sense of the distribution of the data over the range of the plot, much like a histogram.

In the boxplot above, we are interested in whether the residuals look 'heteroskedastic', that is whether the residuals have different variances for men and women. Here, men may have a slightly higher variance, but probably not enough to worry about. We could try adjusting the model to account for heteroskedasticity if we were concerned that it might bias our output.

What is more interesting though, looking at the violinplot is that men have a largely uni-modal distribution, centred over the median, with small bump at the bottom of the plot representing the respondents with very low or zero hours worked in the reference week. However, this is much more pronounced for women who have an effectively bi-model distribution, with the median below the main peak visible in the violin plot.

This makes sense given what we know about working practices, however in terms of our model it may affect our ability to make a good estimate of women's weekly hours.

We can check the normality assumption of the residuals with a quantile-quantile, or q-q plot in statsmodels.

In [ ]:
# Use statsmodels qqplot to graph errors
# make a figure and an axis
f, ax = plt.subplots(figsize=(7,7))
# call the qqplot graph from statsmodels 'graphics' module.
# fits against the normal distribution as standard.
sm.graphics.qqplot(resid, line='45', fit=True, ax=ax);

The q-q plot compares the residuals to a theoretical normal distribution. If the residuals were exactly normally distributed, the blue points would fall along the red line. It is clear that the residuals are broadly normal, except in the left tail. This is because, as we have seen, a disproportionate number of people worked zero or a very low numbers of hours in the reference week, compared to the expectations of a normal distribution.

Currently, the model is underspecified, so before worrying too much about this issue, we would seek to make a better model by adding explanatory variables and see if the issue with residual normality persisted. If it did there are things we might try, like transformations, or other models which treated the hours variable more like a count and adjusted for excess zeroes etc.

## Weighted Regression Model

We can adjust for simple model designs using weights in statsmodels, although it is not sufficient for complex survey designs that involve two stage stratification, clusters etc. As the Labour Force Survey training dataset has provided us with sample weights, we'll use these to produce a weighted regression model. 

In statsmodels, the Weighted Least Squares (WLS) model allows us to fit the same model as in OLS regression with an additional weights parameter.

In [ ]:
# Implement weighted Least squares.
# NB, we might get a warning if some weights are 0, so we'll filter them out first.
lfs_hoursW = lfs_hours[lfs_hours['PWT14R'] > 0] 
weights = lfs_hoursW['PWT14R']
modelW = sm.formula.wls(formula='TOTHRS~SEX',weights=weights, data=lfs_hoursW).fit()
modelW.summary()

Again, we can see that the model coefficients represent the mean hours worked for men (36.4852) and women (27.0043) and are the same as the weighted means calculated earlier (see below).

In [ ]:
# First, group the data including the weight this time.
sexW = lfs_hours.groupby('SEX')[['TOTHRS','PWT14R']]

# iterating over a groupby object produces a tuple: (groupname, sub dataframe)
for group, df in sexW:
    # make DescrStatsW object
    W = sm.stats.DescrStatsW(df['TOTHRS'],df['PWT14R'])
    print(group,W.mean)

Residuals and fitted values can be obtained directly from the weights model, and explored as we did previously.

## Making Predictions

A key part of regression analysis is the ability to make predictions based on the implemented model.

We can do this by specifying the regression equation by extracting the model parameters, and then running new data through the model to get a model prediction (currently in our model, men would get 36.5 hours worked and women 27 hours, as it accounts for nothing other that sex).

However, to make things easier, statsmodels also allows us to `.predict()` the outcome based on inputs in the same form as the model explanatory variables.

In [ ]:
# make a series with two values - Male and Female, with the same variable names as the model.
pSex = pd.Series(data = ['Male','Female'],name='SEX')

# The means for men and women are predicted as expected.
modelW.predict(pSex)

# Multiple Linear Regression

When we add additional variables to a linear regression, such that an outcome is modelled by a combination of two or more explanatory variables, we call it multple regression.

Often, multiple regression allows us to explain a greater amount of the variance in an outcome than is explained by a single explanatory variable, and further, the variance explained by the model can be broken down to see the relative contribution of each explanatory variable.

Let's take a simple example, modelling hours worked with both respondent sex and whether each respondent worked full-time or part-time.

In [ ]:
# filter the lfs data so that we have just those with hours worked, and full and part time (no answer, n=6)
lfs_hrsftpt = lfs_hours[lfs_hours['FTPTWK'] != 'No answer'].copy()
# remove unused categories (otherwise the model will try to include them)
lfs_hrsftpt.loc[:,'FTPTWK'] = lfs_hrsftpt['FTPTWK'].cat.remove_unused_categories()

In [ ]:
formula = 'TOTHRS ~ SEX + FTPTWK'
model2 = sm.formula.ols(formula, data=lfs_hrsftpt).fit()
model2.summary()

In this new model, which explains almost 29% of the variance in weekly hours worked, being in part-time compared to full-time work has a much larger bearing on weekly hours worked than being a female as opposed to a male respondent.

Again, we can intuitively see that these are group means by predicting the outcome for the 4 possible repondents in our model:
* Male, Full-time
* Male, Part-time
* Female, Full-time
* Female, Part-time

In [ ]:
# make a series with two values - Male and Female, with the same variable names as the model.
pSex = pd.Series(data = ['Male','Male','Female','Female'],name='SEX')
pWk = pd.Series(data = ['Full-time', 'Part-time','Full-time', 'Part-time'],name='FTPTWK')
# Make the 2 columns into a dataframe
df = pd.DataFrame([pSex,pWk]).T

df['predictions'] = model2.predict(df)
df

Currently, the model predicts that men in full-time and part-time work have longer work weeks than their female counterparts. However, this model currently treats sex and work-type as independent predictors, whereas we know from our $\chi^{2}$ test earlier that there is an association between sex and full-time and part-time work. This relationship is not being fully realised by this model, which effectively assumes being a man or woman is unrelated to full or part time work.

We can further explore this relationship by adding an interaction term to our model.

In [ ]:
# Note that the colon adds an interaction
formula = 'TOTHRS ~ SEX * FTPTWK'
model3 = sm.formula.ols(formula, data=lfs_hrsftpt).fit()
model3.summary()

The above model, with an interaction between sex and work time, effectively creates new categories which we regress against:
* Male **and** Full-time
* Male **and** Part-time
* Female **and** Full-time
* Female **and** Part-time

You can see how the model addresses these four categories from the output:
* Female and Full-time weekly hours are given by the intercept (34.43)
* Female and Part-time weekly hours are given by the intercept and the Part-time coefficient (34.43 - 17.40 = 17.03)
* Male and Full-time weekly hours are given by the intercept and the Male coefs (34.43 + 5.08 = 39.51)
* Male and Part-time weekly hours are given by the intercept, the Male coef, the Part-time, and Male Part-time coefficients combined (34.43 + 5.08 - 17.40 - 5.49 = 16.62)

This model predicts that being Female **and** Part-time actually means you work slightly longer on average than your male counterpart, but only by about 25 minutes a week.

Incidently, this regression model with an interaction is equivalent to the two variable groupby:

In [ ]:
# groupby on sex and work type.
lfs_hrsftpt.groupby(['SEX','FTPTWK'])['TOTHRS'].mean()

The model with an interaction term only explains about 0.4% more of the variance that the model which treats sex and work-type as independent for something that is a more complex model. However, We can check aic and bic to get an indication of whether the more complex model is better. As the values improve by a fair amount (below), we conclude in this case that the interaction term leads to a better model.

In [ ]:
print("Model 2 AIC:", model2.aic, "Model 3 AIC:", model3.aic)
print("Model 2 BIC:", model2.bic, "Model 3 BIC:", model3.bic)

## Weighted Multivariable Regression

As previously, we can account for weights using the WLS model. Here is an example fitting the intercept model for sex and work-type:

In [ ]:
formula = 'TOTHRS ~ SEX * FTPTWK'
# Filter out zero weighted respondents first.
lfs_hrsftptW = lfs_hrsftpt[lfs_hrsftpt['PWT14R'] > 0]
model3W = sm.formula.wls(formula, weights = lfs_hrsftptW['PWT14R'], data=lfs_hrsftptW).fit()
model3W.summary()

In [ ]:
# Use statsmodels qqplot to graph errors
resid = model3W.resid
# make an axis
f, ax = plt.subplots(figsize=(7,7))
# call the qqplot graph from statsmodels 'graphics' module.
# fits against the normal distribution as standard.
sm.graphics.qqplot(resid, line='45', fit=True, ax=ax);

The residuals of multivariable linear regression models can be assessed in the same way as the examples given for simple linear regression, as above. The current model still doesn't satisfy the assumption that residuals are normally distributed.


# Exercise 3

Use the OLS models to answer the following questions:

1. Evaluate a simple regression model for hours modelled by age group. Age is a categorical variable with many categories, by default it is evaluated against the first category, here: 15-19.
    * Is there evidence for differences in weekly hours by age?
    * Try setting the base category using the formula: 'TOTHRS ~ C(AGEEULR, Treatment(reference="40-44"))'. This makes the reference category: 40-44, the group who work the longest weekly hours on average.
    * What changes in the new model formulation?
    * Try running a model without an intercept 'TOTHRS ~ AGEEULR - 1' what does this tell you?
2. Try specifying a multiple regression model that models weekly hours worked as a combination of: SEX, AGEEULR, ETHUK7R, and FTPTWK.
    * How much of the variation in TOTHRS does this model explain?
    * How many hours would you predict the following people working:
        * A white man, aged 43, in full-time work.
        * An Indian woman, aged 30, in part-time work.
        * A black man, aged 36, in full-time work.
        * A woman of mixed ethnicity, aged 24, in full-time work.
3. Is level of education associated with total hours worked?
    * First filter the data to exclude respondents who 'Don't know' their highest level of education, or gave 'No Answer'.
    * Try a groupby to see if there looks to be any evidence of differences by educational level.
    * Explore the distribution of hours worked by education level further with a box plot.
    * Run a hypothesis test of total weekly hours against education.
        * t-test, only works with two groups, see if you can work out how to do a 'one-way ANOVA.'
        * HINT: First fit an ols model of 'TOTHRS ~ HIQUL15D'
        * Then pass the model to sm.stats.anova_lm(), set the 'typ' parameter to 2.
        * Check the solutions for some interpretation information
    * Finally, explore the previously fitted OLS model for 'TOTHRS ~ HIQUL15D', what does this suggest?

In [ ]:
# Start your code in this cell.


In [ ]:
# Question 1
%load ../Solutions/Statistics/ex3_q1.py


In [ ]:
# Question 2
%load ../Solutions/Statistics/ex3_q2.py


In [ ]:
# Question 3
%load ../Solutions/Statistics/ex3_q3.py
